In [1]:
"""

CMSC 630 - Part 3
Gabriella Graziani

Execution File which processes cell images, performs segmentation, and 
outputs a csv file 'dataset.csv' containing extracted features from images

"""


import cv2
import numpy as np
import matplotlib.pyplot as plt
import random 
import os 
import time
import json
from scipy.signal import convolve2d
import ImageProcessing
import csv
import Classifier

In [2]:
def main(input_file,sp_strength,gauss_noise_strength,gauss_noise_var,median_array,
         lin_fil_w,lin_fil_h,lin_fil_sigma,img_height,img_width,r,erosion_iter,dilation_iter,k):

    image_processing = ImageProcessing.ImageProcessing(img_height,img_width)
    parent_dir = os.getcwd() #get current working directory
    path = os.path.join(parent_dir,input_file) #add input file
    all_images = os.listdir(path) #collect name of all images in file

    #Create output files
    file_list = ['Grayscale','Edges','Erosion','Dilation','GrayscaleSegmentation','ClusterSegmentation']#['Grayscale','SPNoise','GaussianNoise','Histograms',
            #'EqualizedHistograms','LinearFilter','MedianFilter','AveragedHistograms','Edges','Erosion','Dilation','GrayscaleSegmentation','ClusteringSegmentation']
    for item in file_list:
        os.makedirs(item,exist_ok = True)


   
    feature_array = []

    print("BEGINNING: Preprocessing and Feature Extraction\n")
    st_batch = time.time()
    img_tm = []
    index = 0
    for item in all_images:
        st = time.time()
        img_path = 'Cancerous cell smears/'+item
        img = cv2.imread(img_path)
        #img_path = 'EqualizedHistograms/'+item
        print("Preprocessing {}, image {} of {} -- Percent complete: {}%".format(item,index+1,len(all_images),((index+1)/len(all_images)*100)))
        clustered_img = image_processing.clustering(img,2) 
        image_processing.save_img(clustered_img,"ClusterSegmentation/"+item)
        
        bw_img = image_processing.rgb2gray(clustered_img)
        image_processing.save_img(bw_img,"Grayscale/"+item)
        
        back_img = image_processing.cluster_threshold(bw_img)
        #image_processing.save_img(fore_img,"GrayscaleSegmentation/foreground_"+item)
        image_processing.save_img(back_img,"GrayscaleSegmentation/background_"+item)
        
        edge_img = image_processing.find_edges(back_img)
        image_processing.save_img(edge_img,"Edges/"+item)
        
        eros_img = image_processing.erosion(edge_img,r,erosion_iter)
        image_processing.save_img(eros_img,"Erosion/"+item)
        dila_img = image_processing.dilation(eros_img,r,dilation_iter)
        image_processing.save_img(dila_img,"Dilation/"+item)
        
        fore_img = image_processing.extract_features(edge_img,back_img,img)
        
        fore_img.append(item[:-6])
        
        feature_array.append(fore_img)
        index = index+1
    
        
        et = time.time()
        exec = et-st
        img_tm.append(exec)
    avg_tm = sum(img_tm)/len(img_tm)
    et_batch = time.time()
    bw_tm_batch = et_batch-st_batch
    

   # print("PreProcessing and Feature Extraction- Batch Execution time: {} Time per image: {}\n".format(bw_tm_batch,avg_tm))
    
    #print(feature_array)
    labels = ['perimeter','area','mean_r','mean_g','mean_b','max_h_val','name']
    with open('dataset4.csv','w',newline='') as file:
        writer = csv.writer(file)
        writer.writerow(labels)
        for row in feature_array:
            writer.writerow(row)
            
    
  

In [3]:
if __name__ == '__main__':
    with open('config.json','r') as f:
        config = json.load(f)
    main(**config)

BEGINNING: Preprocessing and Feature Extraction

Preprocessing svar02.BMP, image 1 of 499 -- Percent complete: 0.2004008016032064%
Preprocessing svar16.BMP, image 2 of 499 -- Percent complete: 0.4008016032064128%


KeyboardInterrupt: 